In [1]:
import tensorflow as tf
from functools import partial
from libs.pipeline_utils import build_sampler_env, sample
from libs.pipeline_utils import wrap_list_with_score, simple_probability_pipeline, probability, meaning_probability
from libs.utils import load_dictionary, load_transformer, top_best, load_dis

C:\Users\mikhail\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
n_items = 10000
batch_size = 64
enc_seq_len = 64
dec_seq_len = 201

In [3]:
load_dir = "./models/shm_c1"
sampler_args = build_sampler_env(load_dir, batch_size, enc_seq_len, dec_seq_len)

INFO:tensorflow:Restoring parameters from ./models/shm_c1\model.ckpt-58519
INFO:tensorflow:Restoring parameters from ./models/shm_c1\model.ckpt-58519


In [4]:
dictionary = load_dictionary("./data/")
transformer = load_transformer(load_dir)

In [5]:
with tf.device("/cpu:0"):
    dis_be = load_dis('./models/', 'believability')
    dis_st = load_dis('./models/', 'style')

In [6]:
dis_me = load_dis('./models/', 'meaning')

In [7]:
be_f = partial(probability, model=dis_be, transformer=transformer)
st_f = partial(probability, model=dis_st, transformer=transformer)

In [8]:
sample_f = partial(sample, *sampler_args, 
                   dictionary=dictionary, 
                   transformer=transformer, 
                   n_items=n_items,
                   batch_size=64)

In [9]:
seed_phrase = 'Убей себя, урод'
me_f = partial(meaning_probability, model=dis_me, seed_phrase=seed_phrase)

In [10]:
simple_probability_pipeline(seed_phrase, sample_f, [be_f, st_f, me_f], topn=0.02)

C:\Users\mikhail\YandexDisk\Dev\deep_d\actual\libs\meaning_discrimination.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  coef = np.array([1./np.log(1./w2v_model.wv.vocab[word].count) for word in words])
C:\Users\mikhail\YandexDisk\Dev\deep_d\actual\libs\meaning_discrimination.py:47: RuntimeWarning: invalid value encountered in true_divide
  coef /= sum(coef)


[('он пишет:чувствовал.', 0.098834987737129371),
 ('лишние слова, что и сама она напечатана и уверена, что за чтением подлецам все же неправильно, а светил иногда из злобы, раскаивался, что это так непременно очень для меня должно.',
  0.055936522529800012),
 ('надо стараться сумасшедшим.', 0.051420249613737451),
 ('даже это их не мошенники.', 0.049778282348043393),
 ('пьянство это было.', 0.047957591946807446),
 ('за этому месту.', 0.047257938280389648),
 ('я один вступишь в этой идее, в саду.', 0.046059258333114951),
 ('где же проэкзаменовать?', 0.044638611574614566),
 ('самолюбиво.', 0.044521565203133801),
 ('вот во втором этажах, вы видите, вы всето тогда пропасть позволите, возьмете к страдальцу,  долго лжешь, нет, полковник, милостивый государь, никогда вы не могли сообразить этого другого пункта.',
  0.043819063902533373),
 ('вообразите: а он его не пуще любил, он сам пример, ибо все же это его есть!',
  0.042648436197619349),
 ('сперва помирился.', 0.04224706258322071),
 ('этог